# K-Drama Browser - a tool for getting quick summaries of K-Drama's based on user sentiment and reviews

Jeff Barney, MSAI-495: Generative AI Text Project

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

## 0. Parameters <a name="parameters"></a>

In [2]:
VOCAB_SIZE = 100000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

## 1. Load the data <a name="load"></a>

In [3]:
import kagglehub

raw_data_path = kagglehub.dataset_download("chanoncharuchinda/korean-drama-2015-23-actor-and-reviewmydramalist") + "/reviews.csv"

100%|██████████| 8.40M/8.40M [00:00<00:00, 71.1MB/s]

Extracting files...


In [4]:
raw_data_df = pd.read_csv(raw_data_path)
raw_data_df.size

106250

In [5]:
# Filter the dataset, remove colons
cols_to_drop = [col for col in raw_data_df.columns if col not in ['title', 'review_text']]
filtered_data = raw_data_df.drop(columns=cols_to_drop)
filtered_data['review_text'] = filtered_data['review_text'].str.replace(':', '-')

In [6]:
# Combine the review and titles
combined_data = filtered_data.apply(lambda row: 'K-Drama title : ' + ' : K-Drama review : '.join(row.astype(str)), axis=1).tolist()
n_reviews = len(combined_data)
print(f"{n_reviews} reviews loaded")

10625 reviews loaded


In [7]:
combined_data[25]

"K-Drama title : Star Struck : K-Drama review : More like a glimmer rather than being starstruck  I've said it before and I'll say it again- This should have been a movie. Or, at least, been released all at once. The cut-off of many episodes felt unsatisfying and, overall, just got in the way of the flow of the story.Also, it seemed like scenes were missing, and I'm sure a longer runtime would have added more depth to it . . . However--strangely--I'm not mad about it. Maybe it's the indie vibes and solid acting that won me over and made me see past its flaws.Just about Zuho- I do have to bring up how great it was to have him, being the fairly famous K-Pop Idol he is, actually speak out against the hate he was getting for acting in a BL, and not just that, but openly voice his support, love, and respect for the LGBT+ community in his response."

## 2. Tokenize the data <a name="tokenize"></a>

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in combined_data]

In [9]:
# Display an example of a review
text_data[25]

"K - Drama title : Star Struck : K - Drama review : More like a glimmer rather than being starstruck I ' ve said it before and I ' ll say it again - This should have been a movie . Or , at least , been released all at once . The cut - off of many episodes felt unsatisfying and , overall , just got in the way of the flow of the story . Also , it seemed like scenes were missing , and I ' m sure a longer runtime would have added more depth to it . . . However - - strangely - - I ' m not mad about it . Maybe it ' s the indie vibes and solid acting that won me over and made me see past its flaws . Just about Zuho - I do have to bring up how great it was to have him , being the fairly famous K - Pop Idol he is , actually speak out against the hate he was getting for acting in a BL , and not just that , but openly voice his support , love , and respect for the LGBT + community in his response . "

In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: the
4: ,
5: and
6: i
7: a
8: to
9: of


In [15]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(text_data[25])
print(example_tokenised.numpy())

[   27    10    13    63    20   776  3793    20    27    10    13    54
    20    55    34     7 11831   321   122   127 30869     6    11   169
   351    12   261     5     6    11   277   146    12   172    10    18
   204    33   124     7   805     2    66     4    51   322     4   124
  1891    39    51   353     2     3   972    10   197     9   126    96
   140  3872     5     4   163     4    44   173    15     3   108     9
     3  1555     9     3    40     2    82     4    12]


## 3. Create the Training Set <a name="create"></a>

In [16]:
# Create the training set of reviews and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [17]:
example_input_output = train_ds.take(1).get_single_element()

In [18]:
# Example Input
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  27,   10,   13,   63,   20, 2858,   20,   27,   10,   13,   54,
         20,    7,  292,   74,   21,  298,   41,    6,  186,   90,  343,
          3,   64,    2,    6, 1582,    7,  136,    5,    3,  201,  104,
       3486,   16,   86, 2466,    2,   29,    6,  151,   12,   44,   21,
        185,  143,  746,  835,  275,   83,  288,   12,    2,  179,    4,
         75,  149,    6,  515,   62,    7,  910, 1731,  566,   67,  824,
          8, 2602,   79,    7,  339,    4,   18,   64,  583,    3,  360,
         11,   23,   67])>

In [19]:
# Example Output (shifted by one token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  10,   13,   63,   20, 2858,   20,   27,   10,   13,   54,   20,
          7,  292,   74,   21,  298,   41,    6,  186,   90,  343,    3,
         64,    2,    6, 1582,    7,  136,    5,    3,  201,  104, 3486,
         16,   86, 2466,    2,   29,    6,  151,   12,   44,   21,  185,
        143,  746,  835,  275,   83,  288,   12,    2,  179,    4,   75,
        149,    6,  515,   62,    7,  910, 1731,  566,   67,  824,    8,
       2602,   79,    7,  339,    4,   18,   64,  583,    3,  360,   11,
         23,   67,   16])>

## 5. Create the causal attention mask function <a name="causal"></a>

In [20]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## 6. Create a Transformer Block layer <a name="transformer"></a>

In [21]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

## 7. Create the Token and Position Embedding <a name="embedder"></a>

In [22]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

## 8. Build the Transformer model <a name="transformer_decoder"></a>

In [23]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [24]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │    25,620,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 256),    │       658,688 │
│ (TransformerBlock)              │ (None, 2, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 100000)   │    25,700,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,979,168 (198.28 MB)

 Trainable params: 51,979,168 (198.28 MB)

 Non-trainable params: 0 (0.00 B)

## 9. Train the Transformer <a name="train"></a>

In [25]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("K-Drama title", max_tokens=80, temperature=1.0)

In [26]:
# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [ ]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[text_generator],
)

Epoch 1/5
 32/333 ━━━━━━━━━━━━━━━━━━━━ 1:15:03 15s/step - loss: 9.6307

# 3. Generate text using the Transformer

In [ ]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [ ]:
info = text_generator.generate(
    "K-Drama title : Unlock My Boss : K-Drama review : ", max_tokens=80, temperature=1.0
)

In [ ]:
info = text_generator.generate(
    "K-Drama title : Blueming : K-Drama review : ", max_tokens=80, temperature=0.5
)

In [ ]:
info = text_generator.generate(
    "K-Drama title : Star Struck : K-Drama review : ", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)